# SFT LLM and other alignment methods

- SFT to collect clear decision
- alignment to help the reasoning
- Use gpt4 API to generate prompt and completion

```json
[
  {
    "prompt": "she no went to market",
    "completion": "She didn't go to the market."
  }
]
```

- `stabilityai/stablelm-zephyr-3b`

```
<|user|>
List 3 synonyms for the word "tiny"<|endoftext|>
<|assistant|>
1. Dwarf
2. Little
3. Petite<|endoftext|>
```

In [1]:
!git clone --depth 1 https://github.com/mzbac/llama2-fine-tune.git

Cloning into 'llama2-fine-tune'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 1), reused 10 (delta 1), pack-reused 0
Receiving objects: 100% (12/12), 6.36 KiB | 6.36 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [4]:
!pip install -U datasets

  Using cached fsspec-2023.10.0-py3-none-any.whl.metadata (6.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 9.7 MB/s eta 0:00:00ta 0:00:01
Using cached fsspec-2023.10.0-py3-none-any.whl (166 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.5
    Uninstalling datasets-2.14.5:
      Successfully uninstalled datasets-2.14.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmtuner 0.3.2 requires transformers<4.35.0,>=4.31.0, but you have transformers 4.36.1 which is incompatible.


In [1]:
import json
import os
import numpy as np
import pandas as pd

import torch
from datasets import load_dataset
from peft import (
    AutoPeftModelForCausalLM,
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

from utils import find_all_linear_names, print_trainable_parameters

output_dir = "./results"
model_name = "NousResearch/Llama-2-7b-hf"

# dataset = load_dataset("./", data_files="conversations.json", split="train")

2024-01-02 23:21:07.278831: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-02 23:21:07.306499: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/syd/mambaforge/envs/torch-py310/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [7]:
from datasets import load_dataset_builder, list_datasets

In [11]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 26421880/26421880 [00:18<00:00, 1425503.14it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29515/29515 [00:00<00:00, 102390.04it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4422102/4422102 [00:04<00:00, 1025982.69it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5148/5148 [00:00<00:00, 5991197.83it/s]

Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



In [9]:
dataset_list = list_datasets()

/tmp/ipykernel_18017/2270578651.py:1: FutureWarning: list_datasets is deprecated and will be removed in the next major version of datasets. Use 'huggingface_hub.list_datasets' instead.
  dataset_list = list_datasets()


In [10]:
len(dataset_list)

88153

In [16]:
list(filter(lambda n: 'alpaca' in n.lower(), dataset_list))

['huggingnft/alpacadabraz',
 'tatsu-lab/alpaca',
 'crumb/stanford_alpaca_full_prompts',
 'nlpcloud/instructions-dataset-adapted-from-stanford-alpaca-for-gpt-j',
 'dominguesm/alpaca-data-pt-br',
 'Yasbok/Alpaca_arabic_instruct',
 'tbboukhari/Alpaca-in-french',
 'tbboukhari/Alpaca_french_instruct',
 'Bingsu/ko_alpaca_data',
 'bertin-project/alpaca-spanish',
 'NbAiLab/norwegian-alpaca',
 'AndreiMuresanu/alpaca_flan-format',
 'blip-solutions/SlovAlpaca',
 'IlyaGusev/ru_turbo_alpaca',
 'HiTZ/alpaca_mt',
 'cahya/alpaca-id',
 'dvilasuero/somos-alpaca-es',
 'dvilasuero/somos-alpaca-es-rg',
 'somosnlp/somos-alpaca-es',
 'cariai/somos-alpaca-es',
 'dvilasuero/somos-alpaca-es-validations',
 'somosnlp/somos-clean-alpaca-es',
 'cariai/somos-alpaca-es-2',
 'yahma/alpaca-cleaned',
 'nataliaElv/somos-clean-alpaca-es-validations',
 'pszemraj/fleece2instructions-codealpaca',
 'pszemraj/fleece2instructions-inputs-alpaca-cleaned',
 'shibing624/alpaca-zh',
 'QingyiSi/Alpaca-CoT',
 'aia8mason/alpaca',
 'ala

In [20]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [2]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "Deci/DeciLM-7B"
# device = "cuda" # for GPU usage or "cpu" for CPU usage

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", trust_remote_code=True).to(device)

# inputs = tokenizer.encode("In a shocking finding, scientists discovered a herd of unicorns living in", return_tensors="pt").to(device)
# outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_p=0.95)
# print(tokenizer.decode(outputs[0]))

# # The model can also be used via the text-generation pipeline interface
# from transformers import pipeline
# generator = pipeline("text-generation", "Deci/DeciLM-7B", torch_dtype="auto", trust_remote_code=True, device=device)
# outputs = generator("In a shocking finding, scientists discovered a herd of unicorns living in", max_new_tokens=100, do_sample=True, top_p=0.95)
# print(outputs[0]["generated_text"])


In [2]:
import torch
from threading import Event, Thread
from uuid import uuid4
from typing import List, Tuple
import gradio as gr
from transformers import (
    AutoTokenizer,
    StoppingCriteria,
    StoppingCriteriaList,
    TextIteratorStreamer,
)


model_name = "stabilityai/stablelm-zephyr-3b"

tok = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.  Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.\
"""

model_configuration = {
    "start_message": f"<|system|>\n {DEFAULT_SYSTEM_PROMPT }<|endoftext|>",
    "history_template": "<|user|>\n{user}<|endoftext|><|assistant|>\n{assistant}<|endoftext|>",
    "current_message_template": '<|user|>\n{user}<|endoftext|><|assistant|>\n{assistant}',
}
history_template = model_configuration["history_template"]
current_message_template = model_configuration["current_message_template"]
start_message = model_configuration["start_message"]
stop_tokens = model_configuration.get("stop_tokens")
tokenizer_kwargs = model_configuration.get("tokenizer_kwargs", {})

examples = [
    ["Hello there! How are you doing?"],
    ["What is OpenVINO?"],
    ["Who are you?"],
    ["Can you explain to me briefly what is Python programming language?"],
    ["Explain the plot of Cinderella in a sentence."],
    ["What are some common mistakes to avoid when writing code?"],
    [
        "Write a 100-word blog post on “Benefits of Artificial Intelligence and OpenVINO“"
    ],
]

max_new_tokens = 256


class StopOnTokens(StoppingCriteria):
    def __init__(self, token_ids):
        self.token_ids = token_ids

    def __call__(
        self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs
    ) -> bool:
        for stop_id in self.token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False


if stop_tokens is not None:
    if isinstance(stop_tokens[0], str):
        stop_tokens = tok.convert_tokens_to_ids(stop_tokens)

    stop_tokens = [StopOnTokens(stop_tokens)]


def default_partial_text_processor(partial_text: str, new_text: str):
    """
    helper for updating partially generated answer, used by de

    Params:
      partial_text: text buffer for storing previosly generated text
      new_text: text update for the current step
    Returns:
      updated text string

    """
    partial_text += new_text
    return partial_text


text_processor = model_configuration.get(
    "partial_text_processor", default_partial_text_processor
)

def convert_history_to_text(history: List[Tuple[str, str]]):
    """
    function for conversion history stored as list pairs of user and assistant messages to string according to model expected conversation template
    Params:
      history: dialogue history
    Returns:
      history in text format
    """
    text = start_message + "".join(
        [
            "".join(
                [history_template.format(num=round, user=item[0], assistant=item[1])]
            )
            for round, item in enumerate(history[:-1])
        ]
    )
    text += "".join(
        [
            "".join(
                [
                    current_message_template.format(
                        num=len(history) + 1,
                        user=history[-1][0],
                        assistant=history[-1][1],
                    )
                ]
            )
        ]
    )
    return text


def user(message, history):
    """
    callback function for updating user messages in interface on submit button click

    Params:
      message: current message
      history: conversation history
    Returns:
      None
    """
    # Append the user's message to the conversation history
    return "", history + [[message, ""]]


def bot(history, temperature, top_p, top_k, repetition_penalty, conversation_id):
    """
    callback function for running chatbot on submit button click

    Params:
      history: conversation history
      temperature:  parameter for control the level of creativity in AI-generated text.
                    By adjusting the `temperature`, you can influence the AI model's probability distribution, making the text more focused or diverse.
      top_p: parameter for control the range of tokens considered by the AI model based on their cumulative probability.
      top_k: parameter for control the range of tokens considered by the AI model based on their cumulative probability, selecting number of tokens with highest probability.
      repetition_penalty: parameter for penalizing tokens based on how frequently they occur in the text.
      conversation_id: unique conversation identifier.

    """

    # Construct the input message string for the model by concatenating the current system message and conversation history
    messages = convert_history_to_text(history)

    # Tokenize the messages string
    input_ids = tok(messages, return_tensors="pt", **tokenizer_kwargs).input_ids
    if input_ids.shape[1] > 2000:
        history = [history[-1]]
        messages = convert_history_to_text(history)
        input_ids = tok(messages, return_tensors="pt", **tokenizer_kwargs).input_ids
    streamer = TextIteratorStreamer(
        tok, timeout=30.0, skip_prompt=True, skip_special_tokens=True
    )
    generate_kwargs = dict(
        input_ids=input_ids,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=temperature > 0.0,
        top_p=top_p,
        top_k=top_k,
        repetition_penalty=repetition_penalty,
        streamer=streamer,
    )
    if stop_tokens is not None:
        generate_kwargs["stopping_criteria"] = StoppingCriteriaList(stop_tokens)

    stream_complete = Event()

    def generate_and_signal_complete():
        """
        genration function for single thread
        """
        global start_time
        ov_model.generate(**generate_kwargs)
        stream_complete.set()

    t1 = Thread(target=generate_and_signal_complete)
    t1.start()

    # Initialize an empty string to store the generated text
    partial_text = ""
    for new_text in streamer:
        partial_text = text_processor(partial_text, new_text)
        history[-1][1] = partial_text
        yield history


def get_uuid():
    """
    universal unique identifier for thread
    """
    return str(uuid4())


with gr.Blocks(
    theme=gr.themes.Soft(),
    css=".disclaimer {font-variant-caps: all-small-caps;}",
) as demo:
    conversation_id = gr.State(get_uuid)
    gr.Markdown(f"""<h1><center>OpenVINO {model_name} Chatbot</center></h1>""")
    chatbot = gr.Chatbot(height=500)
    with gr.Row():
        with gr.Column():
            msg = gr.Textbox(
                label="Chat Message Box",
                placeholder="Chat Message Box",
                show_label=False,
                container=False,
            )
        with gr.Column():
            with gr.Row():
                submit = gr.Button("Submit")
                stop = gr.Button("Stop")
                clear = gr.Button("Clear")
    with gr.Row():
        with gr.Accordion("Advanced Options:", open=False):
            with gr.Row():
                with gr.Column():
                    with gr.Row():
                        temperature = gr.Slider(
                            label="Temperature",
                            value=0.1,
                            minimum=0.0,
                            maximum=1.0,
                            step=0.1,
                            interactive=True,
                            info="Higher values produce more diverse outputs",
                        )
                with gr.Column():
                    with gr.Row():
                        top_p = gr.Slider(
                            label="Top-p (nucleus sampling)",
                            value=1.0,
                            minimum=0.0,
                            maximum=1,
                            step=0.01,
                            interactive=True,
                            info=(
                                "Sample from the smallest possible set of tokens whose cumulative probability "
                                "exceeds top_p. Set to 1 to disable and sample from all tokens."
                            ),
                        )
                with gr.Column():
                    with gr.Row():
                        top_k = gr.Slider(
                            label="Top-k",
                            value=50,
                            minimum=0.0,
                            maximum=200,
                            step=1,
                            interactive=True,
                            info="Sample from a shortlist of top-k tokens — 0 to disable and sample from all tokens.",
                        )
                with gr.Column():
                    with gr.Row():
                        repetition_penalty = gr.Slider(
                            label="Repetition Penalty",
                            value=1.1,
                            minimum=1.0,
                            maximum=2.0,
                            step=0.1,
                            interactive=True,
                            info="Penalize repetition — 1.0 to disable.",
                        )
    gr.Examples(
        examples, inputs=msg, label="Click on any example and press the 'Submit' button"
    )

    submit_event = msg.submit(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
            temperature,
            top_p,
            top_k,
            repetition_penalty,
            conversation_id,
        ],
        outputs=chatbot,
        queue=True,
    )
    submit_click_event = submit.click(
        fn=user,
        inputs=[msg, chatbot],
        outputs=[msg, chatbot],
        queue=False,
    ).then(
        fn=bot,
        inputs=[
            chatbot,
            temperature,
            top_p,
            top_k,
            repetition_penalty,
            conversation_id,
        ],
        outputs=chatbot,
        queue=True,
    )
    stop.click(
        fn=None,
        inputs=None,
        outputs=None,
        cancels=[submit_event, submit_click_event],
        queue=False,
    )
    clear.click(lambda: None, None, chatbot, queue=False)

demo.queue(max_size=2)
# if you are launching remotely, specify server_name and server_port
#  demo.launch(server_name='your server name', server_port='server port in int')
# if you have any issue to launch on your platform, you can pass share=True to launch method:
# demo.launch(share=True)
# it creates a publicly shareable link for the interface. Read more in the docs: https://gradio.app/docs/
demo.launch(server_name='0.0.0.0')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Running on local URL:  http://0.0.0.0:7861

To create a public link, set `share=True` in `launch()`.


Exception in thread Thread-15 (generate_and_signal_complete):
Traceback (most recent call last):
  File "/home/syd/mambaforge/envs/torch-py310/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/home/syd/mambaforge/envs/torch-py310/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_33615/3155043630.py", line 181, in generate_and_signal_complete
NameError: name 'ov_model' is not defined
Traceback (most recent call last):
  File "/home/syd/mambaforge/envs/torch-py310/lib/python3.10/site-packages/gradio/routes.py", line 488, in run_predict
    output = await app.get_blocks().process_api(
  File "/home/syd/mambaforge/envs/torch-py310/lib/python3.10/site-packages/gradio/blocks.py", line 1431, in process_api
    result = await self.call_function(
  File "/home/syd/mambaforge/envs/torch-py310/lib/python3.10/site-packages/gradio/blocks.py", line 1117, in call_function
    prediction = await utils.

# Build SFT trademark datasets

In [4]:
met_df = pd.read_pickle('../../data-augmentation/train_ds/CODE_cleaned_s41_met_before_cut-off.pickle')
not_met_df = pd.read_pickle('../../data-augmentation/train_ds/CODE_cleaned_s41_not_met_before_cut-off.pickle')

In [5]:
met_df.shape, not_met_df.shape

((36977, 11), (21598, 11))

In [7]:
met_df.head(3)

,appl_number,s_41_requirement_outcome,sub_requirement_name,assessment_summary_txt,word_text_value,date_of_assessment,class_gsi_jsonish,tm_phrase,gs,gs-text-list,gs-nums
170,1739647,MET,NaN,"1. Clear,Exp. Headstart",CROWN RESORTS FOUNDATION,2016-01-04 00:00:00.000,"[[{""41"" : "", Entertainment, Charitable service...",crown resorts foundation,"[{'class': '41', 'text': 'Entertainment'}, {'c...","[Entertainment, Charitable services, namely ed...","(35, 36, 41, 43)"
171,1718503,MET,NaN,"1. Adv.,S44: 1611785, 1651439 2. Adv., deferme...",INFLUX SYSTEMS,2016-01-05 00:00:00.000,"[[{""42"" : "", Outsource service providers in th...",influx systems,"[{'class': '42', 'text': 'Outsource service pr...",[Outsource service providers in the field of i...,"(42,)"
172,1722991,MET,NaN,1. Clear,NUTRIBLITZ,2016-01-07 00:00:00.000,"[[{""7"" : "", Electric blenders for household pu...",nutriblitz,"[{'class': '7', 'text': 'Electric blenders for...","[Electric blenders for household purposes, Ele...","(7,)"


In [8]:
not_met_df.head(3)

,appl_number,s_41_requirement_outcome,sub_requirement_name,assessment_summary_txt,word_text_value,date_of_assessment,class_gsi_jsonish,tm_phrase,gs,gs-text-list,gs-nums
10,1067476,NOT_MET,NaN,1.s41.5-desc 2.with s41;clr,REJUVENATE,2005-10-05 00:00:00.000,"[[{""20"" : "", bolsters, mattresses, pillows, cu...",rejuvenate,"[{'class': '20', 'text': 'bolsters'}, {'class'...","[bolsters, mattresses, pillows, cushions, Furn...","(20,)"
51,1382040,NOT_MET,NaN,"1. s41(5); Section 44: 457529, 1379819",KENT,2010-12-20 00:00:00.000,"[[{""32"" : "", Beer, ""}]]",kent,"[{'class': '32', 'text': 'Beer'}]",[Beer],"(32,)"
63,1446020,NOT_MET,NaN,"1. Exp. S41(5) sesc. Form; Classn, s/s. Sectio...",FASTCARD,2011-09-23 00:00:00.000,"[[{""38"" : "", Point of sale communication servi...",fastcard,"[{'class': '38', 'text': 'Point of sale commun...","[Point of sale communication services, telepho...","(38,)"


In [4]:
s1_template = """ 
trademark "{tm:s}" for goods and services: 
"{gs:s}"
"""

In [13]:
met_1k = met_df.sample(frac=1000/len(met_df))
not_met_1k = not_met_df.sample(frac=1000/len(not_met_df))

merged = pd.concat([met_1k, not_met_1k]).fillna('') # use empty string as the value for nan

merged['s_41_requirement_outcome'].value_counts()

s_41_requirement_outcome
MET        1000
NOT_MET    1000
Name: count, dtype: int64

In [14]:
import json

tm_conversations = []

for r in merged.sample(frac=1.0).itertuples():
    gs_text = '; '.join([item['text'] for item in r.gs])
    prompt = s1_template.format(tm=r.word_text_value, gs=gs_text)

    reason = r.sub_requirement_name if r.sub_requirement_name else 'other'
    completion = f' {r.s_41_requirement_outcome} </s> {reason} '
    tm_conversations.append({'prompt': prompt, 'completion': completion})

with open('tm_conversations.json', 'w') as fh:
    json.dump(tm_conversations, fh, indent=2)

## SFT out of memory errors notes

```plaintext
torch.cuda.OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacty of 23.67 GiB of which 1.38 GiB is free. Including non-PyTorch memory, this process has 22.10 GiB memory in use. Of the allocated memory 18.35 GiB is allocated by PyTorch, and 3.42 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF
  1%|          | 12/1875 [01:32<3:59:11,  7.70s/it]
```

### solution
use `model_name = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'`

## Test datasets

In [2]:
test_df = pd.read_pickle('../train_ds/test_cases_Phrase.pickle')

In [3]:
test_df.head()

,ext_id,tm_number,generic,gs,tm_phrase,Non distinct reason
0,AMCZ-2312189204,2352983,False,"[{'class': '43', 'text': 'Hotel accommodation ...",Casa Renoir,NaN
1,AMCZ-2312189296,2352988,False,"[{'class': '44', 'text': 'Massage'}, {'class':...",Somatic Being,NaN
2,AMCZ-2312189726,2353008,False,"[{'class': '12', 'text': 'Wiper blades for veh...",Dryline,NaN
3,AMCZ-2312190303,2353038,False,"[{'class': '16', 'text': 'Art prints'}, {'clas...",Monsieur Finger,NaN
4,AMCZ-2312190532,2353062,False,"[{'class': '3', 'text': 'Cosmetics for childre...",Ooku and Daimyo,NaN


In [5]:
prompts = []

for r in test_df.itertuples():
    prompts.append(s1_template.format(tm=r.tm_phrase, gs='; '.join([item['text'] for item in r.gs])))
test_df['prompts'] = prompts

In [6]:
test_df.head()

,ext_id,tm_number,generic,gs,tm_phrase,Non distinct reason,prompts
0,AMCZ-2312189204,2352983,False,"[{'class': '43', 'text': 'Hotel accommodation ...",Casa Renoir,NaN,"\ntrademark ""Casa Renoir "" for goods and serv..."
1,AMCZ-2312189296,2352988,False,"[{'class': '44', 'text': 'Massage'}, {'class':...",Somatic Being,NaN,"\ntrademark ""Somatic Being"" for goods and ser..."
2,AMCZ-2312189726,2353008,False,"[{'class': '12', 'text': 'Wiper blades for veh...",Dryline,NaN,"\ntrademark ""Dryline"" for goods and services:..."
3,AMCZ-2312190303,2353038,False,"[{'class': '16', 'text': 'Art prints'}, {'clas...",Monsieur Finger,NaN,"\ntrademark ""Monsieur Finger"" for goods and s..."
4,AMCZ-2312190532,2353062,False,"[{'class': '3', 'text': 'Cosmetics for childre...",Ooku and Daimyo,NaN,"\ntrademark ""Ooku and Daimyo"" for goods and s..."


In [8]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import classification_report, ConfusionMatrixDisplay 

DEV = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model_name ="./merged_peft/final_merged_checkpoint"
adapter_path = "./results/final_checkpoint"
# adapter_path = "./dpo_results/final_checkpoint"

model = AutoPeftModelForCausalLM.from_pretrained(
    adapter_path,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True,
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.bfloat16,
#     load_in_4bit=True,
# )

def generate_output(prompt):
    tokenizer = AutoTokenizer.from_pretrained(adapter_path)
    # tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    inputs = tokenizer.encode(
        f"An AI tool that corrects and rephrase user text grammar errors delimited by triple backticks to standard English.\n### Input: ```{prompt}```\n### Output:",
        return_tensors="pt",
    ).to(DEV)
    
    generate_kwargs = dict(
        input_ids=inputs,
        temperature=0.2,
        top_p=0.95,
        top_k=40,
        max_new_tokens=500,
        repetition_penalty=1.3,
    )
    outputs = model.generate(**generate_kwargs)
    return tokenizer.decode(outputs[0])

In [14]:
p0 = test_df['prompts'].iloc[1]

In [15]:
generate_output(p0).split('Output:')[-1]

' MET 1  ###  NOT_MET  Not_Met  MET 2  ###  Compliance  MET 3  ###  Non_Compliant  MET 4  ###  Invalid  MET 5  ###  Unchanged  MET 6  ###  Meet_Base Standard  MET 7  ###  Meet_Basic  MET 8  ###  Meet the Base  MET 9  ###  Meet the Minimum  MET 10  ###  No_Changes  MET 11  ###  Conformance  MET 12  ###  Conform_To  MET 13  ###  Reported as is (no editing)  MET 14  ###  Original  MET 15  ###  Translation  MET 16  ###  Localisation  MET 17  ###  Generated  MET 18  ###  Language  MET 19  ###  Preparation of legal documents including writing; drafting or recording forms; MET 20  ###  Legal document creation from information provided  MET 21  ###  Provide searchable index of web site contents  MET 22  ###  Indexing of data using online search facilities; Provision of location information via internet website portal  MET 23  ###  Online provision of information relating to entertainment; Providing live performances via video stream; Production of virtual content; Live performance activities; 